In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 25 00:14:35 2018

@author: OZGUR
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from xgboost import XGBRegressor
pd.set_option('display.max_columns', 500)


# reading  from train and test csv
dfz=pd.read_csv("../input/sales_train.csv", sep=',', lineterminator='\n')
df3_test=pd.read_csv("../input/test.csv", sep=',', lineterminator='\n')



#filter out records which has no sale
df=dfz.query('item_cnt_day>0')  # & shop_id==2 & item_id==482')
print(df.head())



 
# total_sales : function for calculate total_sales per month for each shop-item pair.
# date_block_num : a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
group_val=['shop_id']
def total_sales(d,a):
    for i in range(len(a)):
        val=['date_block_num','shop_id','item_id']
        new_column='total_sales_shop'
        aggregated = d.groupby(val).sum() ['item_cnt_day']
        aggregated.name = 'total_sales'
        df5=d[val].join(aggregated,on=val)
        d[new_column]=df5['total_sales']   


df5=pd.DataFrame()    
total_sales(df,group_val)  
dfx=df[['date_block_num','total_sales_shop','shop_id','item_id']].drop_duplicates().copy()
dfx.index=dfx.date_block_num


dfx = dfx.sort_values([ 'shop_id','item_id','date_block_num'])

#Calculation of variables in order to send to our model and get the predictions 
# getting diff between t-1 , t-2 , t-6 , t-12  
melt2 = dfx.copy()
melt2['Last_m_Sales'] = melt2.groupby(['shop_id','item_id'])['total_sales_shop'].shift()
melt2['Last_m_Diff'] = melt2.groupby(['shop_id','item_id'])['Last_m_Sales'].diff()
melt2['Last-1_m_Sales'] = melt2.groupby(['shop_id','item_id'])['total_sales_shop'].shift(2)
melt2['Last-1_m_Diff'] = melt2.groupby(['shop_id','item_id'])['Last-1_m_Sales'].diff()
melt2['Last-6_m_Sales'] = melt2.groupby(['shop_id','item_id'])['total_sales_shop'].shift(6)
melt2['Last-6_m_Diff'] = melt2.groupby(['shop_id','item_id'])['Last-6_m_Sales'].diff()
melt2['Last-12_m_Sales'] = melt2.groupby(['shop_id','item_id'])['total_sales_shop'].shift(12)
melt2['Last-12_m_Diff'] = melt2.groupby(['shop_id','item_id'])['Last-12_m_Sales'].diff()
melt2 = melt2.dropna()
print(melt2)
print(melt2.date_block_num.max())

# divide our train dataset into train and test in order to check our results
train, test = train_test_split(melt2, test_size=0.4)

# setting up predictor variables
predictor_train=train[["shop_id",'item_id',"date_block_num","Last_m_Sales","Last_m_Diff","Last-1_m_Sales","Last-1_m_Diff",
"Last-12_m_Sales","Last-12_m_Diff","Last-6_m_Sales","Last-6_m_Diff"]]
# output variable
outcome_var = train['total_sales_shop']


# calling XGBoost algorithm 
mdl=XGBRegressor(n_estimators =50000,max_depth = 8,eta=0.1)
mdl.fit(predictor_train, outcome_var)

predictor_var_test=test[["shop_id",'item_id',"date_block_num","Last_m_Sales","Last_m_Diff","Last-1_m_Sales","Last-1_m_Diff",
"Last-12_m_Sales","Last-12_m_Diff","Last-6_m_Sales","Last-6_m_Diff"]]
outcome_var2 = test['total_sales_shop']

# predict test output based on the model that is creaated over our train dataset parameters.
predictions = mdl.predict(predictor_var_test)

# check ERROR value
meanSquaredError=mean_squared_error(outcome_var2.round(), predictions.round())
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)

# write restults into file
my_submission = pd.DataFrame( { 'Id': test.shop_id,'item_id': test.item_id,'Week': test.date_block_num,'Sales_Real': outcome_var2 ,'Sales_Guess': predictions.round() } )
my_submission.to_csv('sample_submission2.csv', index=False,sep='\t')
print("Writing complete")
